<a href="https://colab.research.google.com/github/dmat/fakenewsdetection/blob/main/FakeNewsDetection_Glove_LR_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# For local run
data_path = 'data/'

In [40]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/SML/fake news detection/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np  
np.random.seed(0)

from nltk import word_tokenize
from gensim.models import word2vec

import nltk 
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
data_train = pd.read_csv(data_path + 'fnn_train.csv')
data_test = pd.read_csv(data_path + 'fnn_test.csv')

In [4]:
data_train['label_fnn_new'] = np.where(data_train['label_fnn'] == 'fake', 0, 1)
data_test['label_fnn_new'] = np.where(data_test['label_fnn'] == 'fake', 0, 1)

In [ ]:
data_train.head()

,id,date,speaker,statement,sources,paragraph_based_content,fullText_based_content,label_fnn,label_fnn_new
0,3106,2011-01-25T06:00:00-05:00,Joe Wilkinson,A national organization says Georgia has one o...,['http://www.ajc.com/news/georgia-politics-ele...,['A coalition of government watchdog groups la...,A coalition of government watchdog groups last...,fake,0
1,5655,2012-04-02T11:42:20-04:00,Rick Scott,"Says Barack Obama's health care law ""will be t...",['http://www.youtube.com/watch?v=TaC0mKApf9Q&f...,['As Supreme Court justices embarked on three ...,As Supreme Court justices embarked on three da...,fake,0
2,3506,2011-04-01T09:49:05-04:00,J.D. Alexander,Says the Southwest Florida Water Management Di...,['http://www.tampabay.com/news/politics/gubern...,"[""Here's a new one: The Senate budget committe...",Here's a new one: The Senate budget committee ...,fake,0
3,3450,2011-03-21T12:20:02-04:00,Paul Ryan,"""The Congressional Budget Office has this econ...",['http://www.cnn.com/2011/POLITICS/03/17/gop.b...,"['Recently, House Budget chairman Paul Ryan, R...","Recently, House Budget chairman Paul Ryan, R-W...",real,1
4,4776,2011-11-13T07:30:00-05:00,Rodney Frelinghuysen,"Says the Treasury Department ""says 41 percent ...",['http://frelinghuysen.house.gov/index.cfm?sec...,['The millionaires’ tax proposal made its late...,The millionaires’ tax proposal made its latest...,fake,0


In [5]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import spacy
import numpy as np
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_20newsgroups
import string


def mean_transform(X, model):
    transformed = None
    for elem in X:
        aux = ''.join(ch for ch in elem if ch not in set(string.punctuation))
        wordVectors = [np.array(model.get_vector(word)) for word in aux.split(' ') if word in model.vocab]
        if len(wordVectors) == 0:
            length = len(model[next(key for key in model.vocab)])
            wordVectors.append(np.zeros(length))
        meanVector = list(np.mean(wordVectors, axis=0))
        transformed = sparse.vstack([transformed, sparse.coo_matrix(meanVector)])
    return transformed



In [7]:

# glove.42B.300d.txt
glove_input_file = data_path + 'glove.6B.100d.txt'
word2vec_output_file = data_path + 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 100)

In [8]:
word2vec_output_file = data_path + 'glove.6B.100d.txt.word2vec'
model_vanila = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
X_train = mean_transform(data_train['fullText_based_content'], model_vanila)
X_test = mean_transform(data_test['fullText_based_content'], model_vanila)

In [15]:
Y_train = data_train.label_fnn_new
Y_test = data_test.label_fnn_new

In [35]:
# fit logistic regression model
from sklearn.linear_model import LogisticRegression
modelLG = LogisticRegression(solver = 'liblinear',random_state=0)
modelLG.fit(X_train, np.ravel(Y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
from sklearn import metrics
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [36]:
y_pred_lg_train = modelLG.predict(X_train)
y_pred_lg_test = modelLG.predict(X_test)

In [37]:
print("Accuracy is:", metrics.accuracy_score(Y_train, y_pred_lg_train))
print("Mean Squared Error is:", np.sqrt(mean_squared_error(Y_train, y_pred_lg_train)))
print(metrics.classification_report(Y_train, y_pred_lg_train))

Accuracy is: 0.6497501972127268
Mean Squared Error is: 0.5918190625413084
              precision    recall  f1-score   support

           0       0.66      0.62      0.64      7621
           1       0.64      0.68      0.66      7591

    accuracy                           0.65     15212
   macro avg       0.65      0.65      0.65     15212
weighted avg       0.65      0.65      0.65     15212



In [38]:
print("Accuracy is:", metrics.accuracy_score(Y_test, y_pred_lg_test))
print("Mean Squared Error is:", np.sqrt(mean_squared_error(Y_test, y_pred_lg_test)))
print(metrics.classification_report(Y_test, y_pred_lg_test))

Accuracy is: 0.715370018975332
Mean Squared Error is: 0.533507245522184
              precision    recall  f1-score   support

           0       0.58      0.99      0.73       418
           1       0.99      0.53      0.69       636

    accuracy                           0.72      1054
   macro avg       0.79      0.76      0.71      1054
weighted avg       0.83      0.72      0.71      1054



In [34]:
from sklearn.svm import SVC
svc = SVC(kernel='linear', random_state=1)
svc.fit(X_train, np.ravel(Y_train))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [23]:
y_pred_svm_train = svc.predict(X_train)
y_pred_svm_test = svc.predict(X_test)

In [20]:
print("Accuracy is:", metrics.accuracy_score(Y_train, y_pred_svm_train))
print("Mean Squared Error is:", np.sqrt(mean_squared_error(Y_train, y_pred_svm_train)))
print(metrics.classification_report(Y_train, y_pred_svm_train))

Accuracy is: 0.6495529844859321
Mean Squared Error is: 0.5919856548211855
              precision    recall  f1-score   support

           0       0.66      0.61      0.63      7621
           1       0.64      0.69      0.66      7591

    accuracy                           0.65     15212
   macro avg       0.65      0.65      0.65     15212
weighted avg       0.65      0.65      0.65     15212



In [22]:
print("Accuracy is:", metrics.accuracy_score(Y_test, y_pred_svm_test))
print("Mean Squared Error is:", np.sqrt(mean_squared_error(Y_test, y_pred_svm_test)))
print(metrics.classification_report(Y_test, y_pred_svm_test))

Accuracy is: 0.7201138519924098
Mean Squared Error is: 0.529042671254021
              precision    recall  f1-score   support

           0       0.59      0.99      0.74       418
           1       0.99      0.54      0.70       636

    accuracy                           0.72      1054
   macro avg       0.79      0.77      0.72      1054
weighted avg       0.83      0.72      0.72      1054

